In [1]:
import pandas as pd
import cv2
from keras.preprocessing.image import load_img
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from skimage import transform
import numpy as np
import glob


Using TensorFlow backend.
/Users/cmeaton/anaconda3/envs/affective_image/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
# path = '/Users/cmeaton/Documents/code/ds/METIS/sea19_ds7_workingdir/project_5/data/external/'
path = '/Users/cmeaton/Documents/code/ds/METIS/sea19_ds7_workingdir/project_5/data/external/AffectNet/training (1).csv'
df = pd.read_csv(path)


In [3]:
df.head(15)

,subDirectory_filePath,face_x,face_y,face_width,face_height,facial_landmarks,expression,valence,arousal
0,689/737db2483489148d783ef278f43f486c0a97e140fc...,134.0,134.0,899,899,181.64;530.91;188.32;627.82;195.1;723.37;205.2...,1,0.785714,-0.055556
1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,20.0,20.0,137,137,28.82;77.52;29.12;93.25;31.04;108.51;33.03;123...,0,-0.017253,0.004313
2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,11.0,11.0,176,176,30.52;87.33;32.55;106.43;36.94;125.81;43.06;14...,0,0.174603,0.007937
3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,40.0,40.0,269,269,44.43;158.17;47.08;189.2;50.54;221.88;58.3;253...,1,0.153401,0.038890
4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,22.0,22.0,153,153,50.59;78.72;48.6;93.23;48.72;109.06;48.8;123.0...,8,0.783972,-0.551684
5,979/f675c6a88cdef99a6d8b0261741217a0319387fcf1...,89.0,89.0,595,595,124.39;323.36;137.29;395.12;157.18;467.85;179....,6,-0.341270,0.547619
6,637/94b769d8e880cbbea8eaa1350cb8c094a03d27f9fe...,66.0,66.0,471,471,87.68;291.89;96.85;352.15;106.51;410.07;117.81...,9,-2.000000,-2.000000
7,997/b81f843f08ce3bb0c48b270dc58d2ab8bf5bea3e22...,33.0,33.0,226,226,50.26;116.32;51.55;141.85;56.61;167.84;62.87;1...,6,-0.488145,0.831839
8,358/21a32dd1c1ecd57d3e8964621c911df1c0b3348a4a...,111.0,111.0,744,744,190.57;336.55;177.96;420.8;178.66;503.36;189.8...,9,-2.000000,-2.000000
9,330/60216cf5171a1f18876f178e4ab84db22f2dc0920b...,7.0,7.0,214,214,-11.63;81.59;-10.07;120.01;-3.91;156.74;5.25;1...,10,-2.000000,-2.000000


In [4]:
len(df)

414800

In [6]:
# create function to take file_path,
#load image, and return array (224,224,3) + file path into new df...merge expression (class) onto new df

# check function on live_demo.py

In [ ]:
size = 224

def get_input(img_path):
    img = load_img(img_path)
    img = img_to_array(img)
    img = transform.resize(img, (size, size,3), preserve_range=True).astype(np.float32)
    img = preprocess_input(img)
    return [img_path, img]


array_list = []
def path_to_images(df):
    for image in df.loc[1:,'subDirectory_filePath']:
        sub = image.split('/')[0]
        files = glob.glob(f"{sub}/*") #Get list of all images with emotion
        for path in files:
            if path == image:
                array_list.append(get_input(path))
    df_1 = pd.DataFrame(array_list)
    return df_1, array_list

df_1, array_list = path_to_images(df)
        
    

/Users/cmeaton/anaconda3/envs/affective_image/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/cmeaton/anaconda3/envs/affective_image/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
